# Support vector regression

In [87]:
## Dataset
import seaborn as sns
df=sns.load_dataset('tips')

In [88]:
df.shape

(244, 7)

In [89]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [91]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [92]:
df.sex.value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [93]:
df.smoker.value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [94]:
df.day.value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [95]:
df.time.value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [96]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [97]:
df[df.duplicated()]

,total_bill,tip,sex,smoker,day,time,size
202,13.0,2.0,Female,Yes,Thur,Lunch,2


In [98]:
df = df.drop_duplicates().reset_index(drop=True)

In [99]:
df[df.duplicated()]

,total_bill,tip,sex,smoker,day,time,size


In [100]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [101]:
## Independent and Dependent features
X = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y = df['total_bill']

In [102]:
## train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [103]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((182, 6), (182,), (61, 6), (61,))

In [104]:

## ! Let's do feature encoding first. (Label encoding and one hot encoding)

from sklearn.preprocessing import LabelEncoder

In [105]:

# ! If I want to remove unusual warnings then I can use below lines

# import warnings
# warnings.filterwarnings('ignore')

In [106]:
# sex, smoker and time are of binary category

le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

X_train['sex'] = le1.fit_transform(X_train['sex'])
X_train['smoker'] = le2.fit_transform(X_train['smoker'])
X_train['time'] = le3.fit_transform(X_train['time'])

In [107]:
X_train.head()

,tip,sex,smoker,day,time,size
97,1.50,1,1,Fri,0,2
141,6.70,1,0,Thur,1,6
192,2.56,1,1,Thur,1,2
68,2.01,1,0,Sat,0,2
119,2.92,0,0,Thur,1,4


In [108]:
X_test['sex'] = le1.transform(X_test['sex'])
X_test['smoker'] = le2.transform(X_test['smoker'])
X_test['time'] = le3.transform(X_test['time'])

In [109]:
X_test.head()

,tip,sex,smoker,day,time,size
24,3.18,1,0,Sat,0,2
6,2.00,1,0,Sun,0,2
152,2.74,1,0,Sun,0,3
232,1.47,1,0,Sat,0,2
238,5.92,1,0,Sat,0,3


In [110]:

## ! One hot encoding--- Column Transformer.

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [111]:
ct=ColumnTransformer(transformers=[('onehot',OneHotEncoder(drop='first'),[3])],remainder='passthrough')

In [113]:
X_train=ct.fit_transform(X_train)

In [114]:
X_test=ct.transform(X_test)

## Support Vector Regression

In [115]:
from sklearn.svm import SVR
svr = SVR()

In [116]:
svr.fit(X_train, y_train)

SVR()

In [117]:
y_pred = svr.predict(X_test)

In [118]:
from sklearn.metrics import r2_score,mean_absolute_error
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.3966923667178325
4.654096263286009


In [119]:

## ! Hyperparameter Tuning using GridSearch CV
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}


grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.048 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.112 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.008 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.006 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.079 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.060 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.006 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.106 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.142 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.017 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.031 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [120]:
grid.best_params_

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

In [125]:
grid_prediction=grid.predict(X_test)

In [126]:
from sklearn.metrics import r2_score,mean_absolute_error
print(r2_score(y_test,grid_prediction))
print(mean_absolute_error(y_test,grid_prediction))

0.5590597980835699
3.9582781176494692
